In [1]:
import pandas as pd

In [2]:
r_cols = ["user_id","movie_id","rating"]
ratings = pd.read_csv("./ml-100k/u.data", sep="\t", names=r_cols, usecols=range(3))
ratings.head()

,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


In [3]:
import numpy as np

In [4]:
movProps = ratings.groupby("movie_id").agg({"rating": [np.size, np.mean]})

movProps.head()

rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

In [7]:
movieNumRatings = pd.DataFrame(movProps['rating']['size'])
numRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

In [8]:
numRatings.head() #normalised data

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


In [11]:
movieDict = {}
with open(r'ml-100k/u.item', encoding="ISO-8859-1") as f:
    temp = ''
    for line in f:
        #line.decode("ISO-8859-1")
        fields = line.rstrip('\n').split('|')
        movieID = int(fields[0])
        name = fields[1]
        genres = fields[5:25]
        genres = map(int, genres)
        movieDict[movieID] = (name, np.array(list(genres)), numRatings.loc[movieID].get('size'), movProps.loc[movieID].rating.get('mean'))


In [13]:
movieDict[7]

('Twelve Monkeys (1995)',
 array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
 0.6706689536878216,
 3.798469387755102)

In [14]:
from scipy import spatial

def computeDistance(a,b):
    genreA = a[1]
    genreB = b[1]

    genreDistance = spatial.distance.cosine(genreA, genreB)
    popularityA = a[2]
    popularityB = b[2]
    popularityDist = abs(popularityA - popularityB)
    return genreDistance + popularityDist

computeDistance(movieDict[3], movieDict[7])

1.5180102915951972

In [15]:
movieDict[3]

('Four Rooms (1995)',
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
 0.15265866209262435,
 3.033333333333333)

In [16]:
movieDict[7]

('Twelve Monkeys (1995)',
 array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
 0.6706689536878216,
 3.798469387755102)

In [17]:
#Very disimilar movies

In [18]:
import operator

def getNeighbors(movieID, K):
    distances = []
    for movie in movieDict:
        if(movie != movieID):
            dist = computeDistance(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

In [21]:
K = 10
avgRating = 0
neighbors = getNeighbors(2, K)
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print(movieDict[neighbor][0] + " "+str(movieDict[neighbor][3]))

avgRating /= K

Con Air (1997) 3.45985401459854
Clear and Present Danger (1994) 3.569832402234637
Chain Reaction (1996) 2.7
Daylight (1996) 2.7719298245614037
Spawn (1997) 2.6153846153846154
Anaconda (1997) 2.289473684210526
Abyss, The (1989) 3.589403973509934
Lost World: Jurassic Park, The (1997) 2.9430379746835444
Natural Born Killers (1994) 2.953125
Ghost and the Darkness, The (1996) 3.203125


In [22]:
movieDict[2]

('GoldenEye (1995)',
 array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
 0.22298456260720412,
 3.2061068702290076)